In [157]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
dataset = h5py.File('/home/collin/protein_gan/data/dataset.hdf5', 'r')

In [3]:
list(dataset.keys())

['test_128', 'test_16', 'test_64', 'train_128', 'train_16', 'train_64']

In [5]:
test_128 = dataset['test_128']
train_128 = dataset['train_128']
test_64 = dataset['test_64']
train_64 = dataset['train_64']
test_16 = dataset['test_16']
train_16 = dataset['train_16']

In [159]:
n_groups = 2
index = np.arange(n_groups)
bar_width = 0.35

set_16 = [len(test_16), len(train_16)]
set_64 = [len(test_64), len(train_64)]
set_128 = [len(test_128), len(train_128)]

In [172]:
df = pd.DataFrame(test_16)

ValueError: Must pass 2-d input

In [ ]:
df.describe()